### CIRCUITS TRANSFORMATION

In [0]:
from pyspark.sql.functions import col, current_timestamp,round

bronze_table = "keshcatalog.bronze.circuits"
silver_table = "keshcatalog.silver.circuits"

bronze_df = spark.table(bronze_table)

silver_df = (
    bronze_df
    .withColumnRenamed("circuitId", "circuit_id")
    .withColumnRenamed("circuitRef", "circuit_ref")
    .withColumnRenamed("lat", "latitude")
    .withColumnRenamed("lng", "longitude")
    .withColumnRenamed("alt", "altitude")
    .drop("url")
    .withColumn("circuit_id", col("circuit_id").cast("int"))
    .withColumn("latitude",round(col("latitude").cast("double"), 2))
    .withColumn("longitude", round(col("longitude").cast("double"), 2))
    .withColumn("altitude", col("altitude").cast("int"))
    .drop("ingestion_date")
    .dropDuplicates(["circuit_id"])
    .filter(col("circuit_id").isNotNull())
    .orderBy(col("circuit_id").asc())
)

silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(silver_table)


### CONSTRUCTORS TRANSFORMATION

In [0]:
from pyspark.sql.functions import *

bronze_table = "keshcatalog.bronze.constructors"
silver_table = "keshcatalog.silver.constructors"

bronze_df = spark.table(bronze_table)

silver_df = (
    bronze_df
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumnRenamed("constructorRef", "constructor_ref")
    .drop("url")
    .dropDuplicates(["constructor_id"])
    .filter(col("constructor_id").isNotNull())
    .orderBy(col("constructor_id").asc())
)

silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(silver_table)

### RACES TRANSFORMATION

In [0]:
from pyspark.sql.functions import * 

bronze_df=spark.table('keshcatalog.bronze.races_')
races_cleaned_df = bronze_df \
    .withColumn("time", when(col("time") == "\\N", None).otherwise(col("time"))) \
    .withColumn("time", when(length("time") == 5, concat_ws("", col("time"), lit(":00"))).otherwise(col("time")))

silver_df = ((
    races_cleaned_df
    .withColumnRenamed("raceId", "race_id")
    .withColumnRenamed("circuitId", "circuit_id")
    .withColumn(
        "race_timestamp",
            concat_ws(" ", col("date").cast("string"), trim(col("time")))
        )
    )
    .withColumn('race_timestamp',to_timestamp(col("race_timestamp")))
    .drop("url")
    .dropDuplicates(["race_id"])
    .filter(col("race_id").isNotNull())
    .orderBy(col("year").asc(), col("race_id").asc())
    .select(
        "race_id",
        "year",
        "round",
        "circuit_id",
        "name",
        "race_timestamp",
        "ingestion_timestamp"
    )
)


In [0]:
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable('keshcatalog.silver.races_')

### DRIVERS TRANSFORMATION

In [0]:
from pyspark.sql.functions import (
    col,
    concat,
    lit,
    when
)


bronze_table = "keshcatalog.bronze.drivers"
silver_table = "keshcatalog.silver.drivers"

bronze_df = spark.table(bronze_table)

silver_df = (
    bronze_df
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("driverRef", "driver_ref")
    .withColumn("name", concat(col("forename"), lit(" "), col("surname")))
    .withColumn("number", when(col("number") == "\\N", None).otherwise(col("number").cast("int")))
    .select(
        "driver_id",
        "driver_ref",
        "number",
        "code",
        "name",
        "dob",
        "nationality",
        "ingestion_timestamp"
    )
    .dropDuplicates(["driver_id"])
    .filter(col("driver_id").isNotNull())
)

silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(silver_table)

### RESULTS TRANSFORMATION

In [0]:
from pyspark.sql.functions import col, when, round, sum

bronze_table = "keshcatalog.bronze.results"
silver_table = "keshcatalog.silver.results"

bronze_df = spark.table(bronze_table)

silver_df = (
    bronze_df
    .withColumnRenamed("resultId", "result_id")
    .withColumnRenamed("raceId", "race_id")
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumnRenamed("positionText", "position_text")
    .withColumnRenamed("positionOrder", "position_order")
    .withColumnRenamed("fastestLap", "fastest_lap")
    .withColumnRenamed("fastestLapTime", "fastest_lap_time")
    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed")
    .withColumnRenamed("statusId", "status_id")
    
    .withColumn("time", when(col("time") == "\\N", None).otherwise(col("time")) )
    .withColumn("fastest_lap_time", when(col("fastest_lap_time") == "\\N", None).otherwise(col("fastest_lap_time")))
     
    .dropDuplicates(["result_id"])
    .filter(col("result_id").isNotNull())
)

(silver_df.write
 .format('delta')
 .mode('overwrite')
 .option('overwriteSchema','true')
 .saveAsTable('keshcatalog.silver.results') )

In [0]:
from pyspark.sql.functions import count, sum, when

analysis_df = bronze_df.select([
    (sum(when(col(c).isNull(), 1).otherwise(0)) / count("*")).alias(c) for c in bronze_df.columns
])

null_counts_df = silver_df.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in silver_df.columns
])

summary_df = silver_df.describe()
display(null_counts_df)
display(analysis_df)

result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,time,milliseconds,fastest_lap,rank,fastest_lap_time,fastest_lap_speed,status_id,ingestion_timestamp
0,0,0,0,6,0,10735,0,0,0,0,18368,18369,18410,18249,0,18410,0,0


resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,ingestion_timestamp
0.0,0.0,0.0,0.0,2.403846153846154E-4,0.0,0.430088141025641,0.0,0.0,0.0,0.0,0.0,0.7359375,0.7375801282051282,0.7311298076923077,0.0,0.7375801282051282,0.0,0.0


### PIT_STOPS

In [0]:
from pyspark.sql.functions import col, round as spark_round, current_timestamp

pitstops_silver_df = (
    spark.read.table('keshcatalog.bronze.pit_stops')
    .withColumnRenamed('raceId', 'race_id')
    .withColumnRenamed('driverId', 'driver_id')
    .withColumn('duration', spark_round(col('duration').cast('double'), 2).cast('double'))
    .withColumn('ingestion_date', current_timestamp())
)
pitstops_silver_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable('keshcatalog.silver.pit_stops')

### LAP_TIMES

In [0]:
from pyspark.sql.functions import col, current_timestamp, when, trim, split

bronze_table = "keshcatalog.bronze.lap_times"
silver_table = "keshcatalog.silver.lap_times"

bronze_df = spark.table(bronze_table)

silver_df = (
    bronze_df
    .withColumnRenamed("raceId", "race_id")
    .withColumnRenamed("driverId", "driver_id")
    .withColumn("time", trim(col("time")))
    .withColumn(
        "lap_time_seconds",
        when(
            col("time").contains(":"),
            split(col("time"), ":").getItem(0).cast("int") * 60 +
            split(col("time"), ":").getItem(1).cast("double")
        ).otherwise(col("time").cast("double"))
    )
    .withColumn('lap_time_seconds', spark_round(col("lap_time_seconds"), 5))
    .dropDuplicates(["race_id", "driver_id", "lap"])
    .filter(col("race_id").isNotNull() & col("driver_id").isNotNull())
    .orderBy(col("race_id").asc(), col("lap").asc())
    .select('race_id', 'driver_id', 'lap', 'position', 'lap_time_seconds', 'milliseconds', 'ingestion_timestamp')
)
(silver_df.write
 .format('delta')
 .mode('overwrite')
 .option('overwriteSchema','true')
 .saveAsTable(silver_table))

### QUALIFYING 

In [0]:
from pyspark.sql.functions import col, trim, when, split, round as spark_round

bronze_table = "keshcatalog.bronze.qualifying"
silver_table = "keshcatalog.silver.qualifying"

bronze_df = spark.table(bronze_table)

def to_seconds(col_name):
    return when(
        (col(col_name) == "\\N") | (col(col_name) == ""), None
    ).when(
        col(col_name).contains(":"),
        split(col(col_name), ":").getItem(0).cast("int") * 60 +
        split(col(col_name), ":").getItem(1).cast("double")
    ).otherwise(col(col_name).cast("double"))

silver_df = (
    bronze_df
    .withColumnRenamed("qualifyId", "qualify_id")
    .withColumnRenamed("raceId", "race_id")
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumn("q1_seconds", spark_round(to_seconds("q1"), 5))
    .withColumn("q1", when((col("q1") == "\\N") | (col("q1") == ""), None).otherwise(trim(col("q1"))))
    .withColumn("q2_seconds", spark_round(to_seconds("q2"), 5))
    .withColumn("q2", when((col("q2") == "\\N") | (col("q2") == ""), None).otherwise(trim(col("q2"))))
    .withColumn("q3_seconds", spark_round(to_seconds("q3"), 5))
    .withColumn("q3", when((col("q3") == "\\N") | (col("q3") == ""), None).otherwise(trim(col("q3"))))
    .dropDuplicates(["qualify_id"])
    .filter(col("qualify_id").isNotNull() & col("race_id").isNotNull())
    .select(
        "qualify_id", "race_id", "driver_id", "constructor_id", "number", "position",
        "q1_seconds", "q2_seconds", "q3_seconds", "ingestion_timestamp"
    )
    .orderBy(col("race_id").asc(), col("position").asc())
)

silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(silver_table)

### RESULTING TABLE FROM MULTIPLE JOINS

In [0]:
from pyspark.sql.functions import col

# --- 1. Configuration ---
CATALOG_NAME = "keshcatalog"
SILVER_SCHEMA_NAME = "silver"

# --- 2. Read All Necessary Silver Tables ---
print("--- Reading all required Silver tables ---")
try:
    results_df = spark.read.table(f"{CATALOG_NAME}.{SILVER_SCHEMA_NAME}.results")
    races_df = spark.read.table(f"{CATALOG_NAME}.{SILVER_SCHEMA_NAME}.races_")
    drivers_df = spark.read.table(f"{CATALOG_NAME}.{SILVER_SCHEMA_NAME}.drivers")
    constructors_df = spark.read.table(f"{CATALOG_NAME}.{SILVER_SCHEMA_NAME}.constructors")
    circuits_df = spark.read.table(f"{CATALOG_NAME}.{SILVER_SCHEMA_NAME}.circuits")
    print("✅ All silver tables read successfully.")
except Exception as e:
    print(f"❌ ERROR reading silver tables: {e}")
    # dbutils.notebook.exit("Failed to read silver tables") # Uncomment to stop execution on failure

# --- 3. Perform Joins to Create the Gold DataFrame ---
print("\n--- Joining tables to create the race_results_analysis Gold table ---")

# Join all tables in a single chained statement
final_df = results_df \
    .join(races_df, "race_id", "inner") \
    .join(
        drivers_df.withColumnRenamed("name", "driver_name")
                  .withColumnRenamed("nationality", "driver_nationality"),
        "driver_id", "inner"
    ) \
    .join(
        constructors_df.withColumnRenamed("name", "constructor_name")
                       .withColumnRenamed("nationality", "constructor_nationality"),
        "constructor_id", "inner"
    ) \
    .join(
        circuits_df.withColumnRenamed("name", "circuit_name")
                   .withColumnRenamed("location", "circuit_location"),
        "circuit_id", "inner"
    )

# --- 4. Select and Organize Columns for the Final Table ---
# This step creates a clean, user-friendly schema.
race_results_analysis_df = final_df.select(
    col("year").alias("race_year"),
    races_df.name.alias("race_name"),
    col("race_timestamp"),
    col("circuit_name"),
    col("circuit_location"),
    circuits_df.country.alias("circuit_country"),
    col("driver_name"),
    col("driver_nationality"),
    col("constructor_name"),
    col("constructor_nationality"),
    col("grid"),
    col("position"),
    col("points"),
    col("laps"),
    col("time").alias("race_time"),
    col("fastest_lap"),
    col("fastest_lap_time"),
    col("fastest_lap_speed"),
    col("race_id"),
    col("driver_id"),
    col("constructor_id")
)

# --- 5. Write the Analysis Table in silver layer ---
analysis_table = f"{CATALOG_NAME}.{SILVER_SCHEMA_NAME}.race_results_analysis"
print(f"\nWriting analysis table: {analysis_table}")

race_results_analysis_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(analysis_table)

print(f"✅ Success! Created or updated Analysis table: {analysis_table}")

# --- 6. Display a sample of the final data ---
display(race_results_analysis_df.limit(10))



In [0]:
# %sql
# SELECT * FROM keshcatalog.silver.race_results_analysis
# where driver_name = "Ayrton Senna" 



### Driver and Lap_times table joined

In [0]:
lap_times_df = spark.read.table("keshcatalog.silver.lap_times")

driver_lap_performance_df = lap_times_df \
    .join(races_df, "race_id", "inner") \
    .join(drivers_df.withColumnRenamed("name", "driver_name"), "driver_id", "inner") \
    .select(
        col("year"), races_df.name.alias("race_name"),
        col("driver_name"), col("lap"), col("position"),
        col("milliseconds").alias("lap_time_ms"),
        col("race_id"), col("driver_id")
    )

# Write the Gold Table
gold_table_name_2 = "keshcatalog.silver.driver_lap_performance"
driver_lap_performance_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(gold_table_name_2)
print(f"✅ Success! Created or updated driver_lap_performance table: {gold_table_name_2}")


In [0]:
%sql
SELECT * FROM keshcatalog.silver.driver_lap_performance
WHERE race_name = "Austrian Grand Prix" AND year = 2019 AND driver_name = "Lewis Hamilton"

year,race_name,driver_name,lap,position,lap_time_ms,race_id,driver_id
2019,Austrian Grand Prix,Lewis Hamilton,1,3,73741,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,2,3,70327,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,3,3,70182,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,4,3,69867,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,5,3,69611,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,6,3,69666,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,7,3,69689,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,8,3,69724,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,9,3,69845,1018,1
2019,Austrian Grand Prix,Lewis Hamilton,10,3,69807,1018,1


### Pit Stop Analysis Joins

In [0]:
print("\n--- Building Gold table: pit_stop_analysis ---")
pit_stops_df = spark.read.table(f"{CATALOG_NAME}.{SILVER_SCHEMA_NAME}.pit_stops")

pit_stop_analysis_df = pit_stops_df \
    .join(races_df, "race_id", "inner") \
    .join(drivers_df.withColumnRenamed("name", "driver_name"), "driver_id", "inner") \
    .select(
        col("year"), races_df.name.alias("race_name"),
        col("driver_name"), col("stop"), col("lap"), col("time").alias("pit_stop_time_of_day"),
        col("duration").alias("pit_stop_duration_seconds"),
        col("milliseconds").alias("pit_stop_duration_ms"),
        col("race_id"), col("driver_id")
    )

# Write the Gold Table
gold_table_name_3 = "keshcatalog.silver.pit_stop_analysis"
pit_stop_analysis_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(gold_table_name_3)
print(f"✅ Success! Created or updated Gold table: {gold_table_name_3}")

In [0]:
%sql
select * from keshcatalog.silver.pit_stop_analysis
where driver_name = "Lewis Hamilton" and year = 2019

year,race_name,driver_name,stop,lap,pit_stop_time_of_day,pit_stop_duration_seconds,pit_stop_duration_ms,race_id,driver_id
2019,Australian Grand Prix,Lewis Hamilton,1,15,16:35:52,21.51,21515,1010,1
2019,Bahrain Grand Prix,Lewis Hamilton,1,13,18:34:41,24.25,24255,1011,1
2019,Bahrain Grand Prix,Lewis Hamilton,2,34,19:08:53,24.32,24324,1011,1
2019,Chinese Grand Prix,Lewis Hamilton,1,22,14:50:01,22.64,22644,1012,1
2019,Chinese Grand Prix,Lewis Hamilton,2,36,15:13:05,23.6,23597,1012,1
2019,Azerbaijan Grand Prix,Lewis Hamilton,1,13,16:37:17,19.99,19993,1013,1
2019,Spanish Grand Prix,Lewis Hamilton,1,27,15:50:16,22.26,22265,1014,1
2019,Spanish Grand Prix,Lewis Hamilton,2,46,16:16:43,22.51,22511,1014,1
2019,Monaco Grand Prix,Lewis Hamilton,1,11,15:27:03,24.27,24266,1015,1
2019,Canadian Grand Prix,Lewis Hamilton,1,28,14:48:57,23.66,23662,1016,1
